In [1]:
import pandas as pd
from dateutil.relativedelta import relativedelta # finds last day of month
from datetime import timedelta

# https://pynative.com/python-get-last-day-of-month/

In [30]:
# import data
df1 = pd.read_csv('/Users/nicgoebel/Documents/Data Analysis/SQL/Datasets/IRA Contributions/2020 Invesco SIMPLE IRA Activity.csv')
df2 = pd.read_csv('/Users/nicgoebel/Documents/Data Analysis/SQL/Datasets/IRA Contributions/2021 Invesco SIMPLE IRA Activity.csv')
df3 = pd.read_csv('/Users/nicgoebel/Documents/Data Analysis/SQL/Datasets/IRA Contributions/2022 Invesco SIMPLE IRA Activity.csv')

# combine data into one dataframe
df_all = pd.concat([df1, df2, df3]).reset_index(drop=True)

# filter for just contributions
df = df_all.loc[df_all['Transaction'] == 'Contribution']

# convert columns to datetime format
df['trade_date'] = pd.to_datetime(df['trade_date'])
df['pay_period_start'] = pd.to_datetime(df['pay_period_start'])
df['pay_period_end'] = pd.to_datetime(df['pay_period_end'])
df['pay_date'] = pd.to_datetime(df['pay_date'])

# 2022 columns were not named correctly,
# need to combine similar columns ('pay_period_start' & 'Pay Period - Start')
# where index >=108 move bad_columns to good_columns
df.loc[108:, 'trade_date'] = df['Trade Date']
df.loc[108:, 'transaction_amount'] = df['Transaction Amount']
df.loc[108:, 'share_price'] = df['Share Price']
df.loc[108:, 'total_shares'] = df['Total shares']
df.loc[108:, 'pay_period_start'] = df['Pay Period - Start']
df.loc[108:, 'pay_period_end'] = df['Pay Period - End']
df.loc[108:, 'pay_date'] = df['Pay Date']

# drop columns with incorrect names
df.drop(['Trade Date', 'Transaction Amount', 'Share Price', 'Total shares', 'Pay Period - Start', 
         'Pay Period - End', 'Pay Date'], axis=1, inplace=True)

# filter for necessary columns only (don't really need last two sections)
ndf = df[['trade_date', 'pay_date']]

# for contribution_due_date, need to find end of month and add 30 days
for index, row in ndf.iterrows():
    # find last day of month using relativedelta
    ndf.loc[index, 'contribution_due_date'] = (row['pay_date'] + relativedelta(day=31)) + timedelta(days=30)
    
# determine late status
# if trade_date > contribution_due_date THEN late ELSE on-time
ndf.sort_values('pay_date', inplace=True)

for index, row in ndf.iterrows():
    if row['trade_date'] > row['contribution_due_date']:
        ndf.loc[index, 'late_status'] = 'Late'
    else:
        ndf.loc[index, 'late_status'] = 'On-time'

# remove duplicates (employer matches)
ndf.drop_duplicates(keep='first', inplace=True)

ndf

/var/folders/ll/77znv_6d7jng9_kjkpdllhbh0000gn/T/ipykernel_654/4081743280.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['trade_date'] = pd.to_datetime(df['trade_date'])
/var/folders/ll/77znv_6d7jng9_kjkpdllhbh0000gn/T/ipykernel_654/4081743280.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pay_period_start'] = pd.to_datetime(df['pay_period_start'])
/var/folders/ll/77znv_6d7jng9_kjkpdllhbh0000gn/T/ipykernel_654/4081743280.py:15: SettingWithCopyWarning: 
A value is trying to be set on a cop

,trade_date,pay_date,contribution_due_date,late_status
47,2020-01-21,2020-01-21,2020-03-01,On-time
43,2020-02-10,2020-02-06,2020-03-30,On-time
41,2020-03-09,2020-02-21,2020-03-30,On-time
39,2020-06-29,2020-03-06,2020-04-30,Late
37,2020-06-29,2020-03-24,2020-04-30,Late
...,...,...,...,...
117,2022-10-14,2022-09-21,2022-10-30,On-time
115,2022-11-14,2022-10-06,2022-11-30,On-time
112,2022-11-28,2022-10-21,2022-11-30,On-time
111,2022-11-28,2022-11-04,2022-12-30,On-time
